In [76]:
# 类似文本翻译
# 就是输入一个词,然后输出另一个词

In [77]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [78]:
#S代表开始字符  E代表结束字符 ?代表无法识别的字符
letter = [c for c in 'SE?abcdefghijklmnopqrstuvwxyz']
mydata = [['man', 'women'], ['black', 'white'], ['king', 'queen'],
          ['girl', 'boy'], ['up', 'down'], ['high', 'low'],
          ['cat', 'dog'], ['hello', 'world'], ['good', 'bye']]
letter2index = {v: k for k, v in enumerate(letter)}
index2letter = {k: v for k, v in enumerate(letter)}
print(letter2index)
print(index2letter)

{'S': 0, 'E': 1, '?': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{0: 'S', 1: 'E', 2: '?', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


In [79]:
# 每个单词都变为5个字母,不够的在后面补?
max_len = max([max(len(i), len(j)) for i, j in mydata])
print('每个单词变的维度:', max_len)  # 每个单词变的维度: 5
# print('每个单词变的维度:', n_step)  # 每个单词变的维度: 5

每个单词变的维度: 5


In [80]:
n_class = 29  # 26个字母加3个符号(SE?)
batch_size = 1

<img src="encoderdecoder.png"> <br>
- 下面是对数据进行处理，主要做的是
- 首先对单词长度不够的，用 '?' 进行填充
- 然后将 Deocder 的输入数据末尾添加终止标志 'E'
- Decoder 的输入数据开头添加开始标志 'S'
- Decoder 的输出数据末尾添加结束标志 'E'

In [81]:
def make_data(myData):
    encode_input_all = []
    decode_input_all = []
    decode_output_all = []
    # 1.首先对单词长度不够的，用 ‘?’ 进行填充
    for seq in myData:
        for i in range(2):
            seq[i] = seq[i] + '?' * (max_len - len(seq[i]))
        # print(seq)# ['man??', 'women'] ['black', 'white']...['up???', 'down?']
        encode_input = [letter2index[n] for n in (seq[0] + 'E')]
        decode_input = [letter2index[n] for n in ('S' + seq[1])]
        decode_output = [letter2index[n] for n in (seq[1]) + 'E']

        # 用np.eye(n_class) n_class是字典长度 转one-hot
        encode_input_all.append(np.eye(n_class)[encode_input])
        decode_input_all.append(np.eye(n_class)[decode_input])
        decode_output_all.append(decode_output)
    return torch.Tensor(encode_input_all), torch.Tensor(decode_input_all), torch.LongTensor(decode_output_all)



In [82]:
encode_input_all, decode_input_all, decode_output_all = make_data(mydata)

In [83]:
print(encode_input_all, type(encode_input_all), len(encode_input_all))
print("-----------")
print(encode_input_all[1], encode_input_all[1].shape)
#torch.Size([6, 29]) 6代表5+1,,29表示每个字符29维

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0

In [84]:
class myDataset(Data.Dataset):
    def __init__(self, encode_input_all, decode_input_all, decode_output_all):
        self.encode_input_all = encode_input_all
        self.decode_input_all = decode_input_all
        self.decode_output_all = decode_output_all

    def __getitem__(self, item):
        return self.encode_input_all[item], self.decode_input_all[item], self.decode_output_all[item]

    def __len__(self):
        return len(self.encode_input_all)


dataset = myDataset(encode_input_all, decode_input_all, decode_output_all)
# j = 0
# for i in dataset:
#     print(i)
#     j = j + 1
#     print(j)

In [85]:
dataloader = Data.DataLoader(dataset=dataset, batch_size=1, shuffle=False, drop_last=True)

In [86]:
for x, y, z in dataloader:
    print("x", x, x.shape)  # 0 1 2
    print("y", y, y.shape)
    print("z", z, z.shape)
    print('----------------')
    break

x tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]) torch.Size([1, 6, 29])
y tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0.

In [87]:
## model
class Seq2seq(nn.Module):
    def __init__(self):
        super(Seq2seq, self).__init__()
        ##n_hidden = 128
        self.encoder = nn.RNN(input_size=n_class, hidden_size=128, dropout=0.5)
        self.decoder = nn.RNN(input_size=n_class, hidden_size=128, dropout=0.5)
        self.fc = nn.Linear(in_features=64, out_features=n_class)

    def forward(self, encode_input, decode_input):
        batch_size = 1
        encode_input = encode_input.transpose(0, 1)
        decode_input = decode_input.transpose(0, 1)
        h_0 = torch.zeros(1, batch_size, 128)
        out1, h_n1 = self.encoder(encode_input, h_0)
        out2, h_n2 = self.decoder(decode_input, h_n1)
        # out2 = out2[-1]  ## out2本身就只有一个输出
        model = self.fc(out2)
        # print(model.shape)
        return model

In [88]:
model = Seq2seq()
print(model)

Seq2seq(
  (encoder): RNN(29, 64, dropout=0.5)
  (decoder): RNN(29, 64, dropout=0.5)
  (fc): Linear(in_features=64, out_features=29, bias=True)
)


D:\ProgramData\Anaconda3\envs\pycharmshiyong\lib\site-packages\torch\nn\modules\rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [89]:
## 损失函数&&优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(5000):
    for x, y, z in dataloader:
        pred = model(x, y)
        pred = pred.transpose(0, 1)
        # print(pred.shape)
        loss = 0
        for i in range(len(z)):
            loss += loss_fn(pred[i], z[i])
        print(epoch, loss.item())

        ###
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

109 3.3488757610321045
109 3.397664785385132
109 3.3578994274139404
110 3.476572275161743
110 3.371722459793091
110 3.369882583618164
110 3.364511728286743
110 3.4239702224731445
110 3.3896758556365967
110 3.3488757610321045
110 3.397664785385132
110 3.3578994274139404
111 3.476572275161743
111 3.371722459793091
111 3.369882583618164
111 3.364511728286743
111 3.4239702224731445
111 3.3896758556365967
111 3.3488757610321045
111 3.397664785385132
111 3.3578994274139404
112 3.476572275161743
112 3.371722459793091
112 3.369882583618164
112 3.364511728286743
112 3.4239702224731445
112 3.3896758556365967
112 3.3488757610321045
112 3.397664785385132
112 3.3578994274139404
113 3.476572275161743
113 3.371722459793091
113 3.369882583618164
113 3.364511728286743
113 3.4239702224731445
113 3.3896758556365967
113 3.3488757610321045
113 3.397664785385132
113 3.3578994274139404
114 3.476572275161743
114 3.371722459793091
114 3.369882583618164
114 3.364511728286743
114 3.4239702224731445
114 3.3896758

KeyboardInterrupt: 

In [105]:
def test(word):
    encode_input, decode_input, _ = make_data([[word, '?' * len(word)]])
    output = model(encode_input, decode_input)
    # output.data.max(2, keepdim=True)[1]中的2表示，找第3维的最大值；
    # [1]表示，output.data.max(2, keepdim=True)会返回一个数组，
    # 第一个是output数组中第3维度的最大值是多少，
    # 第二个是最大值的位置在哪里。
    predict = output.data.max(2, keepdim=True)[1]
    decode = [letter[i] for i in predict]
    trans = ''.join((decode))
    return trans

In [105]:

print('man->',test('man'))
print('hello->',test('hello'))

In [107]:
for i in mydata[:3]:
    print(i)


['man??', 'women']
['black', 'white']
['king?', 'queen']


<class 'list'>
